<a href="https://colab.research.google.com/github/ganeshdasrla/Assignment/blob/master/OTP_Verification_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##OTP Verification System

Import required libraries

1. Implement a function to generate a 6-digit OTP randomly.

In [44]:
import re
import random
import smtplib
import time
import tkinter as tk
import tkinter.simpledialog
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [27]:
# Global dictionary to store OTPs
otp_store = {}

In [28]:
def generate_otp(length=6):
    otp = ""
    for i in range(length):
      otp += str(random.randint(0,9))
    return int(otp)

In [29]:
generate_otp()

915179

2. Develop a function to simulate sending the OTP to the user's email address.

In [30]:
def send_otp_via_email(recipent_email, otp):
  sender_email = input("Enter your email address: ")
  sender_password = get_email_password(sender_email)

  subject = "Your OTP for verification"
  body = f"Your OTP is: {otp}"

  try:
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipent_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP(host='smtp.gmail.com', port=587)
    server.starttls()
    server.Login(sender_email, sender_password)
    server.sennd_message(msg)
    server.quit()
    print("OTP sent successfully")
  except smtplib.SMTPException as e:
    print("Failed to send OTP:", e)

In [31]:
# Get sender's email password
def get_email_password(email):
  """Prompt user for email password."""
  root = tk.Tk()
  root.withdraw()

  password = simpledialog.askstring("Email Verification", f"Enter password for {email}: ")
  root.destroy()
  return password

In [32]:
def store_otp(recipient, otp, vaild_duration=300):
  """Store OTP along with recipient email and expiry time."""
  expiry_time = time.time() + vaild_duration
  otp_store[recipient] = {'otp': otp, 'expiry': expiry_time}

3. Create a function to prompt the user to enter the OTP received in their email.

In [33]:
def prompt_user_for_otp():
  """Prompt the user to enter the OTP received."""
  root = tk.Tk()
  root.withdraw()

  otp = simpledialog.askstring("OTP Verification", "Enter the OTP received:")
  root.destroy()
  return otp

4. Implement a function to verify if the entered OTP matches the generated OTP.

In [34]:
def verify_otp(recipient, otp_input):
  """Verify if the entered OTP matches the generated OTP."""
  if recipient in otp_store:
    stored_otp = otp_store[recipient]
    if time.time() < stored_otp['expiry']:
      if otp_input == stored_otp['otp']:
        return "OTP verified"
      else:
        return "Invalid OTP"
    else:
      return "OTP has expired"
  else:
    return "Recipient not found"

5. Allow the user to retry OTP entry in case of incorrect input.

In [35]:
def verify_and_prompt(email):
  """Generate OTP, send it via email, prompt user for OTP, and verify."""
  otp = generate_otp()
  send_otp_via_email(email, otp)
  store_otp(email, otp)

  max_attempts = 3
  attempts = 0

  while attempts < max_attempts:
    entered_otp = prompt_user_for_otp()
    if entered_otp is None:
      return

    verification_result = verify_otp(email, entered_otp)
    if verification_result == "OTP verified":
      messagebox.showinfo("OTP Verification", "OTP verified successfully!")
      return
    else:
      attempts += 1
      if attempts < max_attempts:
        messagebox.showerror("OTP Veification", f"Invalid OTP. Attempts left: {max_attempts - attempts}")
      else:
        messagebox.showerror("OTP Verification", "Maximum attempts reached. OTP verification faild, Please try again later.")

In [37]:
# RegEx for validating email address
email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'

In [38]:
def validate_email(email):
  if re.match(email_regex, email):
    return True
  else:
    return False

In [48]:
def main():
  root = tk.Tk() # create main window
  #root.iconify() # minimize main window
  root.withdraw() # hide main window

  email = tkinter.simpledialog.askstring("Email Verification", "Enter your email address:")
  root.destroy()
  if validate_email(email):
    verify_and_prompt(email)

if __name__ == "__main__":
  main()

TclError: no display name and no $DISPLAY environment variable